#  Рекуррентные нейронные сети
В этом блокноте необходимо реализовать  рекуррентные нейронные сети и использовать их для обучения модели, которая может генерировать подписи к изображениям.

Для успешного выполнения блокнота требуется, чтобы была установлена версия scipy не выше 1.2.1. 
Проверьте установленную версию scipy, выполнив код:
import scipy
print(scipy.__version__)

Для установки необходимой версии scipy выполните команду следующую команду (pip/conda):
conda install scipy==1.2.1


In [ ]:
# Необходимые установки
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from dlcv.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from dlcv.rnn_layers import *
from dlcv.captioning_solver import CaptioningSolver
from dlcv.classifiers.rnn import CaptioningRNN
from dlcv.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from dlcv.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# для авто перезагрузки внешних модулей 
# см. http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ возвращает относительную ошибку """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

## Установка h5py

Набор данных COCO, который мы будем использовать, хранится в формате HDF5. Чтобы загружать файлы HDF5, нужно установить пакет `h5py` Python. Из командной строки выполните команду (conda или pip), например: <br/>
`conda install h5py` <br/>

Вы также можете запускать команды среды прямо из  Jupyter notebook, поставив перед командой префикс "!":

Выполните команду, приведенную  ниже в ячейке 

Возможно при установке пакета `h5py` Вы получите сообщения о том,что элементы пакета уже установлены:<br/>
All requested packages already installed.
или
Requirement already satisfied: h5py in c:\users\hp\anaconda3\envs\tf2_env\lib\site-packages (3.7.0)
Requirement already satisfied: numpy>=1.14.5 in c:\users\hp\anaconda3\envs\tf2_env\lib\site-packages (from h5py) (1.19.2)

In [ ]:
!conda install h5py

# Microsoft COCO
В этом задании мы будем использовать  набор данных Microsoft COCO (http://mscoco.org/) 2014 года, который стал стандартным испытательным набором для изображений с субтитрами. Набор данных состоит из 80 000 обучающих изображений и 40 000 проверочных изображений, каждое из которых снабжено 5 подписями.

Объем данных COCO составляет ~ 1 ГБ. Он должен быть загружен в  каталог `dlcv/datasets/coco_captioning` в предварительно обработанном виде (см.ниже). 

Для этого надо скачать файл `train_val_VGG_pcah5.zip`  и разархивировать его в указанный каталог. Если у Вас возникнут затруднения с загрузкой набора данных COCO, обратитесь к  преподавателю, чтобы скопировать набор данных и разместить его в указанном каталоге вручную.

Для всех изображений набора данных COCO  были извлечены признаки на уровне fc7 для сети VGG-16, предварительно обученной на данных ImageNet. Чтобы уменьшить время обработки и снизить требования к памяти,  размерность признаков  была сокращена с 4096 до 512 путем сжатия с использованием метода главных компонент; эти признаки были сохранены в файлах `train2014_vgg16_fc7_pca.h5` и `val2014_vgg16_fc7_pca.h5`, которые и находятся в архиве `train_val_VGG_pcah5.zip`.

Исходные изображения занимают в памяти много места (около 20 ГБ), поэтому они не были включены в архив. Все изображения взяты из системы Flickr, а URL-адреса обучающих и проверочных изображений хранятся в файлах `train2014_urls.txt` и` val2014_urls.txt`, соответственно. Это позволяет загружать изображения на лету для визуализации. Поскольку изображения загружаются на лету, **вы должны быть подключены к Интернету для просмотра изображений**.

Работа со строками неэффективна, поэтому мы будем работать с кодированной версией подписи к изображению. Каждому слову присваивается целочисленный идентификатор, что позволяет нам представлять подписи изображений последовательностью целых чисел. Таблица соответствия между целочисленными идентификаторами и словами находится в файле `coco2014_vocab.json`. Вы можете использовать функцию `decode_captions` из файла `dlcv / coco_utils.py` для преобразования массивов целочисленных идентификаторов обратно в строки.

Есть пара специальных токенов (лексем), которые добавлены в словарь. Это специальный токен `<START>` и  токен `<END>`, которые добавляются в начало и конец каждого заголовка изображения, соответственно. Редкие слова заменяются специальным токеном `<UNK>`  ("unknown"- «неизвестно»). Кроме того, поскольку при обучении используются  мини-блоки, содержащие заголовки различной длины, то короткие подписи дополняются специальным токеном `<NULL>` после токена `<END>` и   для `<NULL>` токенов не вычисляются потери или градиенты.  Так как обработка специальных токенов создает некоторые сложности, то Вам предлагается готовое решение, которое избавит Вас от необходимости писать соответствующий код.

Вы можете загрузить все данные MS-COCO (заголовки, функции, URL-адреса и словарь), используя функцию `load_coco_data` из файла` dlcv/ coco_utils.py`. Запустите ниже ячейку блокнота, чтобы сделать это:

In [ ]:
# Загрузка данных COCO c диска; функция возвращает словарь
# В этом блокноте будем работать  только c сокращенной размерностью признаков (512), т.е. флаг pca_features=True 

data = load_coco_data(pca_features=True)

# Печать названий ключей и типов значений из словаря данных
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

## Просмотр данных

Полезно взглянуть на примеры из набора данных, прежде чем начать работу с ним.

Вы можете использовать функцию `sample_coco_minibatch` из файла` dlcv / coco_utils.py` для выборки мини-блоков данных из структуры данных, возвращаемой  `load_coco_data`. Выполните ячейку блокнота ниже, чтобы выбрать небольшой миниблок обучающих данных и показать изображения и подписи к ним. Повторное выполнение ячейки поможет Вам получить более полное представление о наборе данных.

Обратите внимание, что  заголовки  декодируются с помощью  функцию `decode_captions`. Так как загрузка изображений происходит на лету по URL-адресам в системе Flickr, то **Вы должны быть подключены к Интернету для просмотра изображений**.

In [ ]:
# Выборка миниблока, отображение изображений и заголовков

batch_size = 3
# Извлекаем массивы заголовков, признаков изображений, url адресов
captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)

# Для каждого изображения мини-блока
for i, (caption, url) in enumerate(zip(captions, urls)):
    # Отображаем изображение по url адресу
    plt.imshow(image_from_url(url))
    plt.axis('off')
    #Декодируем заголовок изображения 
    caption_str = decode_captions(caption, data['idx_to_word'])
    plt.title(caption_str)
    plt.show()

#  Рекуррентные нейросети

Для формирования подписей к изображениям  будем использовать  рекуррентную нейронную сеть (RNN). Файл `dlcv / rnn_layers.py` содержит реализации слоев различных типов, которые необходимы для построения RNN, а файл` dlcv / classifiers / rnn.py` использует эти слои для построения модели нейросети для снабжения изображений подписями.

Сначала реализуем различные типы слоев RNN в файле `dlcv / rnn_layers.py`.

# RNN: шаг прямого распространения
Откройте файл `dlcv / rnn_layers.py`. Функции в этом файле предназначены для реализации прямого и обратного распространения для разных типов слоев, которые обычно используются в рекуррентных нейронных сетях.

Сначала реализуйте функцию `rnn_step_forward`, которая выполняет прямое распространение для одного временного шага  рекуррентной нейронной сети. После этого выполните следующую ячейку, чтобы проверить  реализацию. Ошибки должны быть на уровне e-8 или меньше.

In [ ]:
N, D, H = 3, 10, 4

x = np.linspace(-0.4, 0.7, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.2, 0.5, num=N*H).reshape(N, H)
Wx = np.linspace(-0.1, 0.9, num=D*H).reshape(D, H)
Wh = np.linspace(-0.3, 0.7, num=H*H).reshape(H, H)
b = np.linspace(-0.2, 0.4, num=H)

next_h, _ = rnn_step_forward(x, prev_h, Wx, Wh, b)
expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

print('ошибка next_h: ', rel_error(expected_next_h, next_h))

# RNN: шаг обратного распространения

В файле `dlcv / rnn_layers.py` реализуйте функцию` rnn_step_backward`. После этого выполните ячейку ниже, чтобы проверить градиенты, вычисляемые Вашей реализацией функции. Ошибки должны быть на уровне «e-8» или менее.

In [ ]:
from dlcv.rnn_layers import rnn_step_forward, rnn_step_backward
np.random.seed(231)
N, D, H = 4, 5, 6
x = np.random.randn(N, D)
h = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_step_forward(x, h, Wx, Wh, b)

dnext_h = np.random.randn(*out.shape)

fx = lambda x: rnn_step_forward(x, h, Wx, Wh, b)[0]
fh = lambda prev_h: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_step_forward(x, h, Wx, Wh, b)[0]
fb = lambda b: rnn_step_forward(x, h, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dnext_h)
dprev_h_num = eval_numerical_gradient_array(fh, h, dnext_h)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dnext_h)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dnext_h)
db_num = eval_numerical_gradient_array(fb, b, dnext_h)

dx, dprev_h, dWx, dWh, db = rnn_step_backward(dnext_h, cache)

print('ошибка dx: ', rel_error(dx_num, dx))
print('ошибка dprev_h: ', rel_error(dprev_h_num, dprev_h))
print('ошибка dWx: ', rel_error(dWx_num, dWx))
print('ошибка dWh: ', rel_error(dWh_num, dWh))
print('ошибка db: ', rel_error(db_num, db))

# RNN: прямое распространение
Теперь, когда вы реализовали прямое и обратное распространение для одного шага обычной RNN, необходимо объединить эти части для реализации RNN, которая обрабатывает всю последовательность данных.

В файле `dlcv / rnn_layers.py` реализуйте функцию` rnn_forward`. Она должно быть реализована с помощью функции `rnn_step_forward`, которую вы определили выше. После этого выполните ячейку ниже, чтобы проверить Вашу реализацию. Вы должны увидеть ошибки порядка «e-7» или менее.

In [ ]:
N, T, D, H = 2, 3, 4, 5

x = np.linspace(-0.1, 0.3, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.3, 0.1, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.4, num=D*H).reshape(D, H)
Wh = np.linspace(-0.4, 0.1, num=H*H).reshape(H, H)
b = np.linspace(-0.7, 0.1, num=H)

h, _ = rnn_forward(x, h0, Wx, Wh, b)
expected_h = np.asarray([
  [
    [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
    [-0.39525808, -0.22554661, -0.0409454,   0.14649412,  0.32397316],
    [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525],
  ],
  [
    [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
    [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
    [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]])
print('ошибка h: ', rel_error(expected_h, h))

# RNN: обратное распространение
В файле `dlcv / rnn_layers.py` реализуйте обратное распространение для RNN в виде функции` rnn_backward`. Она  должна выполнять обратное распространение по всей последовательности входных данных с использованием функции `rnn_step_backward`, которую вы определили ранее. Если функция уже реализована, то ознакомьтесь с её реализацией. Вы должны увидеть ошибки порядка e-6 или меньше.

In [ ]:
np.random.seed(231)

N, D, T, H = 2, 3, 10, 5

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = rnn_backward(dout, cache)

fx = lambda x: rnn_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: rnn_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: rnn_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print('ошибка dx: ', rel_error(dx_num, dx))
print('ошибка dh0: ', rel_error(dh0_num, dh0))
print('ошибка dWx: ', rel_error(dWx_num, dWx))
print('ошибка dWh: ', rel_error(dWh_num, dWh))
print('ошибка db: ', rel_error(db_num, db))

# Слой встраивания (эмбединга) слов: прямое распространение

В системах глубокого обучения мы обычно представляем слова, используя векторы. Каждое слово словаря будет ассоциировано с вектором, и эти векторные представления будут обучаться совместно с остальной частью системы.

В файле `dlcv / rnn_layers.py` реализуйте функцию `word_embedding_forward` для преобразования слов (представленных целыми числами) в векторы. Если функция уже реализована, то ознакомьтесь с её реализацией. Запустите ячейку ниже, чтобы проверить реализацию. Вы должны увидеть ошибку порядка e-8 или меньше.

In [ ]:
N, T, V, D = 2, 4, 5, 3

x = np.asarray([[0, 3, 1, 2], [2, 1, 0, 3]])
W = np.linspace(0, 1, num=V*D).reshape(V, D)

out, _ = word_embedding_forward(x, W)
expected_out = np.asarray([
 [[ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.42857143,  0.5,         0.57142857]],
 [[ 0.42857143,  0.5,         0.57142857],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429]]])

print('ошибка out: ', rel_error(expected_out, out))

# Слой встраивания (эмбединга) слов: обратное распространение
Реализуйте обратное распространение для слоя встраивания слов в функции `word_embedding_backward`. Если функция уже реализована, то ознакомьтесь с её реализацией. После этого выполните ячейку ниже, чтобы численно проверить градиент . Вы должны увидеть ошибку порядка «e-11» или менее.

In [ ]:
np.random.seed(231)

N, T, V, D = 50, 3, 5, 6
x = np.random.randint(V, size=(N, T))
W = np.random.randn(V, D)

out, cache = word_embedding_forward(x, W)
dout = np.random.randn(*out.shape)
dW = word_embedding_backward(dout, cache)

f = lambda W: word_embedding_forward(x, W)[0]
dW_num = eval_numerical_gradient_array(f, W, dout)

print('ошибка dW: ', rel_error(dW, dW_num))

# Временной аффинный слой

На каждом временном шаге мы используем аффинную функцию для преобразования выходного скрытого вектора RNN в рейтинги каждого слова из словаря. Поскольку это очень похоже на аффинный слой полносвязных сетей, мы предоставили Вам готовую реализацию функций `temporal_affine_forward` и` temporal_affine_backward` в файле `dlcv / rnn_layers.py`. Выполните ячейку ниже, чтобы выполнить проверку с помощью численного градиента. Вы должны увидеть ошибки порядка e-9 или меньше.

In [ ]:
np.random.seed(231)

# Проверка градиента для временного аффинного слоя
N, T, D, M = 2, 3, 4, 5
x = np.random.randn(N, T, D)
w = np.random.randn(D, M)
b = np.random.randn(M)

out, cache = temporal_affine_forward(x, w, b)

dout = np.random.randn(*out.shape)

fx = lambda x: temporal_affine_forward(x, w, b)[0]
fw = lambda w: temporal_affine_forward(x, w, b)[0]
fb = lambda b: temporal_affine_forward(x, w, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dw_num = eval_numerical_gradient_array(fw, w, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

dx, dw, db = temporal_affine_backward(dout, cache)

print('ошибка dx: ', rel_error(dx_num, dx))
print('ошибка dw: ', rel_error(dw_num, dw))
print('ошибка db: ', rel_error(db_num, db))

# Временные  Softmax потери

В языковой RNN модели  на каждом временном шаге  производится  оценка рейтинга для каждого слова из словаря. Мы знаем истинное слово  на каждом временном шаге, поэтому  используем softmax функцию потерь  для вычисления потерь и градиента на каждом временном шаге. Потери суммируются во времени и усредняются по мини-пакетам.

Однако есть одна проблема: поскольку мы работаем с мини-пакетами, а разные заголовки изображений могут иметь разную длину, то необходимо добавлять токены (лексемы) `<NULL>` в конце каждой подписи, чтобы они все имели одинаковую длину. Мы не хотим, чтобы токены `<NULL>`   учитывались при вычислении потерь или градиентов, поэтому в дополнение к рейтингам и меткам истинных слов наша функция потерь принимает массив `mask`, который сообщает, какие элементы рейтингов учитываются в потерях.

Определение временной softmax функции потерь соответствует обычной функции кросс-энтропийных потерь, которую Вы реализовывали ранее. Мы написали эту функцию потерь для вас; просмотрите код функции `temporal_softmax_loss` в файле` dlcv / rnn_layers.py`.

Выполните ячейку ниже, чтобы проверить правильность вычисления потерь и  градиента. Вы должны увидеть ошибку для dx порядка e-7 или меньше.

In [ ]:
# Проверка для временной softmax функции потерь
from dlcv.rnn_layers import temporal_softmax_loss

N, T, V = 100, 1, 10

def check_loss(N, T, V, p):
    x = 0.001 * np.random.randn(N, T, V)
    y = np.random.randint(V, size=(N, T))
    mask = np.random.rand(N, T) <= p
    print(temporal_softmax_loss(x, y, mask)[0])
  
check_loss(100, 1, 10, 1.0)   # Должно быть около 2.3
check_loss(100, 10, 10, 1.0)  # Должно быть около 2.3
check_loss(5000, 10, 10, 0.1) # Должно быть около 2.3

# Проверка градиента для временной softmax функции потерь
N, T, V = 7, 8, 9

x = np.random.randn(N, T, V)
y = np.random.randint(V, size=(N, T))
mask = (np.random.rand(N, T) > 0.5)

loss, dx = temporal_softmax_loss(x, y, mask, verbose=False)

dx_num = eval_numerical_gradient(lambda x: temporal_softmax_loss(x, y, mask)[0], x, verbose=False)

print('ошибка dx: ', rel_error(dx, dx_num))

# RNN для формирования заголовков изображений

Теперь, когда вы реализовали необходимые слои, вы можете объединить их, чтобы создать модель, формирующую заголовки изображений. Откройте файл `dlcv / classifiers / rnn.py` просмотрите класс` CaptioningRNN`.

Реализуйте прямое и обратное распространение в функции `loss`. На данный момент Вам нужно только реализовать случай, когда `cell_type = 'rnn'` для обычной RNN. Если в файле код уже реализован, то просто ознакомьтесь с ним. После этого выполните ячейку ниже, чтобы проверить  прямое распространение, используя небольшой тестовый пример; Вы должны увидеть ошибку порядка «e-10» или менее.

In [ ]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(word_to_idx,
          input_dim=D,
          wordvec_dim=W,
          hidden_dim=H,
          cell_type='rnn',
          dtype=np.float64)

# Задание значений всех параметров модели 
for k, v in model.params.items():
    model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)
# Сформируем мини-блок данных
features = np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D)
captions = (np.arange(N * T) % V).reshape(N, T)
# Проверка метода loss
loss, grads = model.loss(features, captions)
expected_loss = 9.83235591003

print('потери: ', loss)
print('ожидаемые потери: ', expected_loss)
print('разность: ', abs(loss - expected_loss))

Запустите  ячейку ниже, чтобы выполнить проверку численного градиента для класса `CaptioningRNN`; Вы должны увидеть ошибки порядка «e-6» или меньше.

In [ ]:
np.random.seed(231)

batch_size = 2
timesteps = 3
input_dim = 4
wordvec_dim = 5
hidden_dim = 6
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
vocab_size = len(word_to_idx)
# Сформируем мини-блок данных
captions = np.random.randint(vocab_size, size=(batch_size, timesteps))
features = np.random.randn(batch_size, input_dim)

model = CaptioningRNN(word_to_idx,
          input_dim=input_dim,
          wordvec_dim=wordvec_dim,
          hidden_dim=hidden_dim,
          cell_type='rnn',
          dtype=np.float64,
        )

loss, grads = model.loss(features, captions)

for param_name in sorted(grads):
    f = lambda _: model.loss(features, captions)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print('%s относительная ошибка: %e' % (param_name, e))

#  Переобучение на данных малого объема

В этом задании используется класс `CaptioningSolver` для обучения моделей, формирующих подписи к изображениям. Откройте файл `dlcv / captioning_solver.py` и просмотрите код класса `CaptioningSolver`; он должен быть для Вас знакомым.

После того, как вы ознакомитесь с API класса, выполните ячеку ниже, чтобы убедиться, что  модель переобучается на небольшом наборе из 100 обучающих примеров. Окончательные потери  должны быть менее 0,1.

In [ ]:
np.random.seed(231)

small_data = load_coco_data(max_train=50)

small_rnn_model = CaptioningRNN(
          cell_type='rnn',
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          hidden_dim=512,
          wordvec_dim=256,
        )

small_rnn_solver = CaptioningSolver(small_rnn_model, small_data,
           update_rule='adam',
           num_epochs=50,
           batch_size=25,
           optim_config={
             'learning_rate': 5e-3,
           },
           lr_decay=0.95,
           verbose=True, print_every=10,
         )

small_rnn_solver.train()

# Построение графика функции потерь на этапе обучения
plt.plot(small_rnn_solver.loss_history)
plt.xlabel('Итерации')
plt.ylabel('Потери')
plt.title('История потерь при обучении')
plt.show()

#  Тестирование модели
Модели, формирующие  подписи ведут себя по-разному во время обучения и во время тестирования. Во время обучения у нас есть доступ к истинному заголовку, поэтому мы вводим истинные слова в качестве входных данных для RNN на каждом этапе. Во время тестирования мы предсказваем слово на отдельном временном шаге и подаем это слово на вход RNN на следующем временном шаге.

В файле `dlcv / classifiers / rnn.py` реализуйте метод` sample` обеспечивающий тестирование RNN. Если в файле код уже реализован, то просто ознакомьтесь с ним. После этого выполните ячейку ниже, чтобы получить заголовки на выходе переобученной модели как при  обучении, так и при проверке. Выборки на обучающих данных должны быть очень хорошими; выборки на проверочных данных, вероятно, не будут иметь смысла.

In [ ]:
for split in ['train', 'val']:# Проверка на 2-х мн-вах данных: обучающих и тестовых
    #Формируем мини-блоки (2 примера) из обучающих и затем из тестовых данных 
    minibatch = sample_coco_minibatch(small_data, split=split, batch_size=2)
    # Извлекаем правильные заголовки (gt), признаки изображений, url адреса
    gt_captions, features, urls = minibatch
    #Декодируем правильные заголовки 
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])
    #Выполняем предсказание заголовков с помощью модели сети
    sample_captions = small_rnn_model.sample(features)
    #Декодируем предсказанные заголовки 
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])
    #Отображаем результаты
    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        # Отображаем изображение по url адресу
        plt.imshow(image_from_url(url))
        # Название этапа, предсказанный заголовок, правильный заголовок
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()

#  ВОПРОС 1


В текущей реализации языковая модель RNN генерирует слово на каждом временном шаге в качестве своего выхода. Однако альтернативный способ постановки проблемы состоит в том, чтобы научить сеть работать с буквами (например, 'a', 'b' и т. Д.), а не словами, чтобы на каждом шаге по времени она получала предыдущий символ в качестве входных данных и пыталась предсказать следующий символ в последовательности. Например, сеть может генерировать заголовок изображения как

'A', ' ', 'c', 'a', 't', ' ', 'o', 'n', ' ', 'a', ' ', 'b', 'e', 'd '

Можете ли вы указать преимущество модели, которая использует RNN на уровне символов для формирования подписи к изображениям? Можете ли вы указать недостаток? СОВЕТ: есть несколько правильных ответов, но может быть полезно сравнить пространство параметров моделей на уровне слов и на уровне символов.

**Ваш ответ:**
